In [ ]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
!pip install sweetviz
import sweetviz
import lightgbm as lgbm
import xgboost as xgb
import imblearn
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
encoder= ce.OrdinalEncoder(cols=['gender','Partner','Dependents','PhoneService','MultipleLines','InternetService',
                                 'OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV',
                                 'StreamingMovies','Contract','PaperlessBilling','PaymentMethod','Churn'],return_df=True,
mapping=[{'col':'gender','mapping':{'Female':0,'Male':1}},
         {'col':'Partner','mapping':{'No':0,'Yes':1}},
         {'col':'Dependents','mapping':{'No':0,'Yes':1}},
         {'col':'PhoneService','mapping':{'No':0,'Yes':1}},
         {'col':'InternetService','mapping':{'No':0,'DSL':1,'Fiber optic':2}},
         {'col':'MultipleLines','mapping':{'No phone service':0,'No':1,'Yes':2}},
         {'col':'OnlineSecurity','mapping':{'No internet service':0,'No':1,'Yes':2}},
         {'col':'OnlineBackup','mapping':{'No internet service':0,'No':1,'Yes':2}},
         {'col':'DeviceProtection','mapping':{'No internet service':0,'No':1,'Yes':2}},
         {'col':'TechSupport','mapping':{'No internet service':0,'No':1,'Yes':2}},
         {'col':'StreamingTV','mapping':{'No internet service':0,'No':1,'Yes':2}},
         {'col':'StreamingMovies','mapping':{'No internet service':0,'No':1,'Yes':2}},
         {'col':'Contract','mapping':{'Month-to-month':0,'One year':1,'Two year':2}},
         {'col':'PaperlessBilling','mapping':{'No':0,'Yes':1}},
         {'col':'PaymentMethod','mapping':{'Mailed check':0,'Electronic check':1,'Bank transfer (automatic)':2,
                                           'Credit card (automatic)':3}},
         {'col':'Churn','mapping':{'No':0,'Yes':1}}])
df_main = encoder.fit_transform(df)

In [ ]:
df_main["TotalCharges"] = pd.to_numeric(df_main["TotalCharges"],errors = 'coerce', downcast="float")

In [ ]:
df_main = df_main.drop(['customerID'],axis = 1)

In [ ]:
df_main.info()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer()
df_main = pd.DataFrame(imputer.fit_transform(df_main),columns = df_main.columns)

In [ ]:
df_main

In [ ]:
y = df_main['Churn']
x = df_main.drop(['Churn'],axis = 1)

In [ ]:
sme = imblearn.combine.SMOTEENN(random_state=45,n_jobs = -1)
x, y = sme.fit_resample(x,y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 45)

In [ ]:
model = xgb.XGBClassifier(n_estimators = 1000,n_jobs = -1, verbose = 2, random_state = 45)
model.fit(x_train,y_train)

In [ ]:
y_pred = model.predict(x_test)
print(f1_score(y_test,y_pred,average = 'weighted'))
print(accuracy_score(y_test,y_pred))